In [1]:
from pyconnect.spark import SparkCluster
sparkcluster = SparkCluster().getOrCreate(
  'spark-cluster-pyconnect', 
  minExecutors=2,
  executorMemory = "4g",
  driverMemory = "4g" )

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import trim

from pyspark.sql import SQLContext

from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer
from pyspark.ml.clustering import KMeansModel
from pyspark.ml import Pipeline

import pandas as pd

hc = sparkcluster.session


SparkSession is created.
Check http://xhadrevrm2p.aetna.com:10001 for spark job UI

The application ID for the spark job is application_1598803737406_922342
If you exit without stopping the spark session, use the following command to kill the job:
yarn application -kill application_1598803737406_922342


Member_Base

In [3]:
mem = hc.table('cust_exp_enc.n323489_Med_mem').toPandas()

Navigator Information

In [4]:
nav = hc.table('cust_exp_enc.n323489_search').toPandas()

In [5]:
nav_mem = hc.table("cust_exp_enc.n323489_search_mem").toPandas()

Call Info

In [6]:
call = hc.table('cust_exp_enc.n323489_calls').toPandas()

Claims Info

In [7]:
clm = hc.table("""cust_exp_enc.n323489_clm_data""").select('paid_amt','clm_ln_status_cd','survey_month','member_id','claim_line_id','srv_spclty_ctg_cd_desc','srv_start_dt').toPandas()

Outlier Treatment for Paid_amt

In [8]:
clm['paid_amt']= clm['paid_amt'].apply(pd.to_numeric)

import numpy as np
per_95 = np.nanpercentile(clm['paid_amt'], 95)

udf = lambda x : per_95 if x>= per_95 else x 
clm['paid_amt']= clm['paid_amt'].apply(udf)

Subscriber Level Call and Navigator Info 



In [9]:
nav_sub = hc.table('cust_exp_enc.n323489_search_sub').toPandas()
nav_mem_sub = hc.table("cust_exp_enc.n323489_search_mem_sub").toPandas()
call_sub = hc.table('cust_exp_enc.n323489_calls_sub').toPandas()


# Validation of MEDS self-reported data 

# Website Search Question 

In [10]:
website = mem[(~mem['used_website'].isnull() )]    #not null website search ques

Grouping by the response 

In [11]:
web = pd.DataFrame(website.groupby(['used_website','survey_month']).member_id.nunique()).reset_index()
web['prop_ques_"used_website"_response'] = ((web['member_id']/web['member_id'].sum())*100).round(2)
web.rename(columns = {'member_id' : 'cnt_members'},inplace=True)

looking at clickstream data for these mem

In [12]:

web_yes_nav = pd.merge(website,nav,on = ['member_id','survey_month'], how = 'inner')

abc_no_sub = pd.DataFrame(web_yes_nav.groupby(['survey_month','used_website']).member_id.nunique()).reset_index()
abc_no_sub.rename(columns = {'member_id' : 'nav_login_no_sub'},inplace=True)


looking at clickstream data for the subscribers 

In [13]:
web_yes_nav_sub = pd.merge(website,nav_sub,on = ['member_id','survey_month'], how = 'inner')

abc_sub = pd.DataFrame(web_yes_nav_sub.groupby(['survey_month','used_website']).member_id.nunique()).reset_index()
abc_sub .rename(columns = {'member_id' : 'nav_login_sub'},inplace=True)



In [14]:
abc = pd.merge(abc_no_sub,abc_sub,on = ['used_website','survey_month'],how='left')
abc['nav_login'] = abc['nav_login_no_sub'] + abc['nav_login_sub']
abc = abc[['survey_month' , 'used_website','nav_login']]

In [15]:
web = pd.merge(web,abc,on = ['used_website','survey_month'],how='left')
web.fillna(0,inplace=True)
web['prop_actually_used_web'] = ((web['nav_login']/web['cnt_members'])*100).round(2)

In [16]:
udf = lambda x : 'Yes' if x==1 else 'No'
web['used_website'] = web['used_website'].apply(udf)
web.rename(columns = {'used_website' : 'ques_used_website_response' , 'nav_login' : 'cnt_mbr_nav_login' },inplace=True)

In [17]:
web

,ques_used_website_response,survey_month,cnt_members,"prop_ques_""used_website""_response",cnt_mbr_nav_login,prop_actually_used_web
0,Yes,2020-08-01,7480,62.06,3621,48.41
1,No,2020-08-01,4572,37.94,616,13.47


# CALLED CSR

In [18]:
called = mem[(~mem['called_csr'].isnull() )]

In [19]:
call_csr = pd.DataFrame(called.groupby(['called_csr','survey_month']).member_id.nunique()).reset_index()
call_csr['prop_ques_"called_csr"_response'] = ((call_csr['member_id']/call_csr['member_id'].sum())*100).round(2)
call_csr.rename(columns = {'member_id' : 'cnt_members'},inplace=True)

Joining with ASD Calls data 

In [20]:
calls_no_sub = pd.merge(call,called,on =['member_id'],how='inner')

abcd_no_sub = pd.DataFrame(calls_no_sub.groupby(['called_csr']).member_id.nunique()).reset_index()
abcd_no_sub.rename(columns = {'member_id' : 'called_aetna_no_sub'},inplace=True)


In [21]:
calls_sub = pd.merge(call_sub,called,on =['member_id'],how='inner')

abcd_sub = pd.DataFrame(calls_sub.groupby(['called_csr']).member_id.nunique()).reset_index()
abcd_sub.rename(columns = {'member_id' : 'called_aetna_sub'},inplace=True)


In [22]:
abcd = pd.merge(abcd_no_sub,abcd_sub,on = ['called_csr'],how='left')
abcd['called_aetna'] = abcd['called_aetna_sub'] + abcd['called_aetna_no_sub']
abcd = abcd[['called_csr' ,'called_aetna']]

In [23]:
call_csr = pd.merge(call_csr,abcd,on = ['called_csr'],how='left')
call_csr.fillna(0,inplace=True)
call_csr['prop_called_aetna'] = ((call_csr['called_aetna']/call_csr['cnt_members'])*100).round(2)

In [24]:
call_csr['called_csr'] = call_csr['called_csr'].apply(udf)
call_csr.rename(columns = {'called_csr' : 'ques_called_csr_response' , 'called_aetna' : 'cnt_mbr_called_aetna' },inplace=True)

In [25]:
calls = pd.concat([calls_sub,calls_no_sub])

Reason for Call

In [26]:
 calls.dropna(subset=['service_request_type_description','service_request_subtype_description'],inplace=True)

In [27]:
reason_call= pd.DataFrame(calls.groupby(['service_request_type_description','survey_month_x']).member_id.nunique()).reset_index()
Top_reason = reason_call.sort_values(ascending = False, by = (['survey_month_x','member_id'])).head(10)
Top_reason.rename(columns = {'member_id' : 'cnt_members','survey_month_x': 'survey_month'},inplace=True)

In [28]:
reason_call= pd.DataFrame(calls.groupby(['service_request_subtype_description','survey_month_x']).member_id.nunique()).reset_index()
Top_sub_reason = reason_call.sort_values(ascending = False, by = (['survey_month_x','member_id'])).head(10)
Top_sub_reason.rename(columns = {'member_id' : 'cnt_members','survey_month_x': 'survey_month'},inplace=True)

# Each MIP Task

Putting all the tasks in a list

In [29]:
list_1 = ['doctor_search','review_coverage','received_eob','reviewed_claim','filled_prescription','called_csr','used_website']

For each task, looking at Yes-No response proportion

In [30]:
d={'response' :[ 1,2]}
task_response = pd.DataFrame(d)
for i in range(0,7):
        abc = pd.DataFrame(mem.groupby(list_1[i]).member_id.nunique()).reset_index()  
        abc.rename(columns = {list_1[i] : 'response' , 'member_id' : list_1[i]+'_mbr_cnt'},inplace=True)
        abc[list_1[i]+'_mbr_cnt_prop'] = ((abc[list_1[i]+'_mbr_cnt']/abc[list_1[i]+'_mbr_cnt'].sum())*100).round(2)
        task_response = pd.merge(task_response,abc)
        
task_response['response'] = task_response['response'].apply(udf)
date = pd.DataFrame([mem.survey_month.unique()]*len(task_response),columns = {'survey_month'})
task_response['survey_month'] = date['survey_month']


In [31]:
task_response

,response,doctor_search_mbr_cnt,doctor_search_mbr_cnt_prop,review_coverage_mbr_cnt,review_coverage_mbr_cnt_prop,received_eob_mbr_cnt,received_eob_mbr_cnt_prop,reviewed_claim_mbr_cnt,reviewed_claim_mbr_cnt_prop,filled_prescription_mbr_cnt,filled_prescription_mbr_cnt_prop,called_csr_mbr_cnt,called_csr_mbr_cnt_prop,used_website_mbr_cnt,used_website_mbr_cnt_prop,survey_month
0,Yes,4898,40.64,6496,53.9,8902,73.86,7001,58.09,10213,84.74,5000,41.49,7480,62.06,2020-08-01
1,No,7154,59.36,5556,46.1,3150,26.14,5051,41.91,1839,15.26,7052,58.51,4572,37.94,2020-08-01


# MIP Tasks

In [32]:
TB = lambda x : '  Top' if x == 5 else '  Bottom'

Grouping by all tasks and looking at members with different customer satisfaction scores

In [33]:
def cs(sat_metric):   
    query1="""
    select {0} ,survey_month,
    count(distinct case when customer_satisfaction = 1 then member_id end  ) as CSAT_not_at_all_satisfied,
    count(distinct case when customer_satisfaction = 2 then member_id end ) as CSAT_not_too_satisfied,
    count(distinct case when customer_satisfaction = 3 then member_id end ) as CSAT_somewhat_satisfied,
    count(distinct case when customer_satisfaction = 4 then member_id end  ) as CSAT_very_satisfied,
    count(distinct case when customer_satisfaction = 5 then member_id end ) as CSAT_completely_satisfied
    from cust_exp_enc.n323489_Med_mem
    where {0} in (1,5)
    group by {0},survey_month
    """.format(sat_metric) 
    test=hc.sql(query1)
    test1=test.toPandas()
    test1['satisfaction'] = test1[sat_metric].apply(TB)
    test1['question'] =  [sat_metric,sat_metric]    +test1['satisfaction']
    #test1['question'] =  test1[sat_metric].astype('str')+  [' ',' '] +  [sat_metric,sat_metric] 
    test1.drop([sat_metric],axis = 1,inplace = True) 
    return test1 

# Key Metrices

In [34]:
def func(question) :
    
    #taking Top bottom satisfaction members
    a = [1.0,5.0]
    doc = mem[mem[question].isin(a)]     

    mem_doc = pd.DataFrame(doc[question].value_counts()).reset_index()
    mem_doc.rename(columns = {'index' : question, question :'cnt_mbrs'},inplace= True)

    #Customer Satisfaction  

    cs = pd.DataFrame(doc.groupby([question]).customer_satisfaction.mean().round(2)).reset_index()
    cs.rename(columns = {'customer_satisfaction' : 'avg_customer_satisfaction_score'},inplace=True)

    doctor = pd.merge(mem_doc,cs,on = [question])
    
    doctor['satisfaction'] = doctor[question].apply(TB)
    doctor['question'] =   [question,question] + doctor['satisfaction']
    #doctor['question'] = doctor[question].astype('str') + [' ',' '] + [question,question]

    doctor['survey_month'] = date['survey_month']
    doctor = doctor[['question',question,'survey_month','cnt_mbrs','avg_customer_satisfaction_score']]
    #Navigator Login     

    doc_nav = pd.merge(doc,nav_mem,on = ['member_id'], how ='inner')

    login_cnt = pd.DataFrame(doc_nav.groupby([question]).login_count.sum()).reset_index()
    login_cnt.rename(columns = {'member_id' :'number_of_logins'},inplace=True)

    login_mem = pd.DataFrame(doc_nav.groupby([question]).member_id.nunique()).reset_index()
    login_mem.rename(columns = {'member_id' :'mbrs_nav_login'},inplace=True)

    login_con = pd.merge(login_mem,login_cnt,on = [question])
    login_con['avg_login_per_mbr'] = (login_con['login_count']/login_con['mbrs_nav_login']).round(2)

    doctor = pd.merge(doctor,login_con,on = [question])
    
    #Claims Data

    doc_clm = pd.merge(doc,clm,on = ['member_id'], how ='inner')
    doc_clm['concat'] = doc_clm.member_id.astype(str) + doc_clm.srv_start_dt.astype(str)

    clm_cnt_paid = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'P'].groupby([question]).concat.nunique()).reset_index()
    clm_cnt_paid.rename(columns = {'concat' :'total_paid_clm'},inplace=True)

    clm_mem_paid = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'P'].groupby([question]).member_id.nunique()).reset_index()
    clm_mem_paid.rename(columns = {'member_id' :'mbrs_clm_paid'},inplace=True)

    clm_con_paid = pd.merge(clm_mem_paid,clm_cnt_paid,on = [question])
    clm_con_paid['avg_paid_clm_per_mem'] = (clm_con_paid['total_paid_clm']/clm_con_paid['mbrs_clm_paid']).round(2)
    
    clm_cnt_den = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'D'].groupby([question]).concat.nunique()).reset_index()
    clm_cnt_den.rename(columns = {'concat' :'total_den_clm'},inplace=True)

    clm_mem_den = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'D'].groupby([question]).member_id.nunique()).reset_index()
    clm_mem_den.rename(columns = {'member_id' :'mbrs_clm_den'},inplace=True)

    clm_con_den = pd.merge(clm_mem_den,clm_cnt_den,on = [question])
    clm_con_den['avg_den_clm_per_mem'] = (clm_con_den['total_den_clm']/clm_con_den['mbrs_clm_den']).round(2)
    
    clm_concat = pd.merge(clm_con_den ,clm_con_paid)
    
    paid_amt_avg = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'P'].groupby([question]).paid_amt.mean().round(2)).reset_index()
    paid_amt_avg.rename(columns = {'paid_amt' :'avg_paid_amt'},inplace=True)
    
    paid_amt_median = pd.DataFrame(doc_clm[doc_clm['clm_ln_status_cd'] == 'P'].groupby([question]).paid_amt.median().round(2)).reset_index()
    paid_amt_median.rename(columns = {'paid_amt' :'median_paid_amt'},inplace=True)
    
    paid_con = pd.merge(paid_amt_avg ,paid_amt_median)
    
    clm_con = pd.merge(clm_concat ,paid_con)
    
    doctor = pd.merge(doctor,clm_con, on = [question])
    #Calls

    doc_call = pd.merge(doc,call,on = ['member_id' ], how ='inner')

    call_cnt = pd.DataFrame(doc_call.groupby([question]).event_identifier.nunique()).reset_index()
    call_cnt.rename(columns = {'event_identifier' :'total_calls'},inplace=True)

    call_mem = pd.DataFrame(doc_call.groupby([question]).member_id.nunique()).reset_index()
    call_mem.rename(columns = {'member_id' :'total_mem_called'},inplace=True)

    call_con = pd.merge(call_mem,call_cnt,on = [question])
    call_con['avg_call_per_mem'] = (call_con['total_calls']/call_con['total_mem_called']).round(2)

    doctor = pd.merge(doctor,call_con, on = [question])
    doctor.drop([question],axis = 1,inplace = True)
    return doctor

# Specialities

In [35]:
def speciality_top(question) :
    
        #taking Top bottom satisfaction members
        a = [1.0,5.0]
        doc = mem[mem[question].isin(a)]     
        
        #Claims Data - Top specialities
        clm_paid= clm[clm['clm_ln_status_cd'] == 'P']
        
        doc_clm = pd.merge(doc,clm_paid,on = ['member_id'], how ='inner')

        bottom = pd.DataFrame(doc_clm[doc_clm[question]== 1.0].groupby(['srv_spclty_ctg_cd_desc','survey_month_x']).member_id.nunique()).reset_index()
        bottom.rename(columns = {'survey_month_x': 'survey_month','member_id' : 'bottom_mbr_cnt'},inplace=True)

        top = pd.DataFrame(doc_clm[doc_clm[question]== 5.0].groupby(['srv_spclty_ctg_cd_desc','survey_month_x']).member_id.nunique()).reset_index()
        top.rename(columns = {'survey_month_x': 'survey_month','member_id' : 'top_mbr_cnt'},inplace=True)

        specialities = pd.merge(top,bottom,how='outer')
        specialities['total_mem'] = specialities['bottom_mbr_cnt'] + specialities['top_mbr_cnt']

        specialities['bottom_prop'] = ((specialities['bottom_mbr_cnt']/specialities['total_mem'])*100).round(2)

        sum_mbr = (doc.member_id.nunique())*.02

        specialities = specialities[specialities['total_mem'] >= sum_mbr]
        specialities.sort_values(ascending = False,by = 'bottom_prop',inplace=True)
        specialities.rename(columns = {'bottom_prop' : 'bottom_prop' + '_' + str(question)},inplace=True)
        return specialities[['survey_month','srv_spclty_ctg_cd_desc','bottom_prop' + '_' + str(question)]].head(10)

Putting all the data into different dataframes

In [36]:
c = []
speciality = pd.DataFrame(columns = speciality_top('sat_doctor_search').columns)

abc = ['sat_doctor_search', 'sat_review_coverage', 'sat_received_eob','sat_reviewed_claim', 'sat_filled_prescription', 'sat_called_csr','sat_used_website']
for i in abc :
    abcd = speciality_top(i)
    speciality = pd.merge(speciality,abcd,how = 'outer')


In [37]:
c = []
df = pd.DataFrame(columns = func('sat_doctor_search').columns)

abc = ['sat_doctor_search', 'sat_review_coverage', 'sat_received_eob','sat_reviewed_claim', 'sat_filled_prescription', 'sat_called_csr','sat_used_website']
for i in abc :
    abcd = func(i)
    df = pd.concat([df,abcd])

In [38]:
c = []
satisfaction = pd.DataFrame(columns = cs('sat_doctor_search').columns)

abc = ['sat_doctor_search', 'sat_review_coverage', 'sat_received_eob','sat_reviewed_claim', 'sat_filled_prescription', 'sat_called_csr','sat_used_website']
for i in abc :
    abcd = cs(i)
    satisfaction = pd.concat([satisfaction,abcd])
    

Storing these into temp tables in hive

In [39]:
#----First Tab

web = hc.createDataFrame(web)
call_csr = hc.createDataFrame(call_csr)
Top_reason = hc.createDataFrame(Top_reason)
Top_sub_reason = hc.createDataFrame(Top_sub_reason)

#--Second Tab 

task_response = hc.createDataFrame(task_response)
satisfaction = hc.createDataFrame(satisfaction)

#--Third Tab

df = hc.createDataFrame(df)

#--Fouth Tab

speciality = hc.createDataFrame(speciality)

web.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_web')
call_csr.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_call_csr')
Top_reason.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_top_reason')
Top_sub_reason.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_top_sub_reason')

task_response.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_task_response')
satisfaction.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_satisfaction')

df.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_question')
speciality.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_speciality')


In [40]:
web.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_web_temp')

In [41]:
call_csr.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_call_csr_temp')

In [42]:
Top_reason.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_top_reason_temp')

In [43]:
Top_sub_reason.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_top_sub_reason_temp')

In [44]:
task_response.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_task_response_temp')

In [45]:
satisfaction.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_satisfaction_temp')

In [46]:
df.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_question_temp')

In [47]:
speciality.write.mode("overwrite").saveAsTable('cust_exp_enc.n323489_speciality_temp')

Insert these temp tables into hive

In [48]:
def data_ingest_to_hive(insert_table,drop_table): 
    query1="""
        insert into table {0}
        Select  * from {1}
        """.format(insert_table,drop_table)
    hc.sql(query1)

In [49]:
data_ingest_to_hive('cust_exp_enc.n323489_web','cust_exp_enc.n323489_web_temp')
data_ingest_to_hive('cust_exp_enc.n323489_call_csr','cust_exp_enc.n323489_call_csr_temp')
data_ingest_to_hive('cust_exp_enc.n323489_top_reason','cust_exp_enc.n323489_top_reason_temp')
data_ingest_to_hive('cust_exp_enc.n323489_top_sub_reason','cust_exp_enc.n323489_top_sub_reason_temp')

data_ingest_to_hive('cust_exp_enc.n323489_task_response','cust_exp_enc.n323489_task_response_temp')
data_ingest_to_hive('cust_exp_enc.n323489_satisfaction','cust_exp_enc.n323489_satisfaction_temp')

data_ingest_to_hive('cust_exp_enc.n323489_question','cust_exp_enc.n323489_question_temp')

data_ingest_to_hive('cust_exp_enc.n323489_speciality','cust_exp_enc.n323489_speciality_temp')

Now loading these tables back here to export in excel

In [32]:
list2 = ['web','call_csr','top_reason','top_sub_reason','task_response','question','satisfaction','speciality']
for i in range(0,8):
    list2[i] = hc.table('cust_exp_enc.n323489_' + str(list2[i])).toPandas()
    
for i in range(5,7):
    list2[i].sort_values(by = ['question','survey_month'],inplace = True)
    
for i in range(0,2):
    list2[i].sort_values(by = ['survey_month'],inplace = True,ascending= [True])
    
for i in range(2,4):
    list2[i].sort_values(by = ['survey_month','cnt_members'],inplace = True,ascending= [True,False])
    
list2[4].sort_values(by = ['survey_month'],inplace = True,ascending= [True])
list2[7].sort_values(by = ['survey_month'],inplace = True,ascending= [True])

list2 -- list of dataframes

# OVERALL LEVEL ANALYSIS

#Used Website

used_website = pd.DataFrame(list2[0].copy().groupby('ques_used_website_response').sum()).reset_index()

used_website['prop_ques_"used_website"_response'] = ((used_website['cnt_members']/used_website['cnt_members'].sum())*100).round(2)
used_website['prop_actually_used_web'] = ((used_website['cnt_mbr_nav_login']/used_website['cnt_members'])*100).round(2)

col1 = list(list2[0].columns)
col1.remove('survey_month')

used_website = used_website[col1]

#called csr

called_csr = list2[1].copy()

called_csr['called'] =  (called_csr['prop_ques_"called_csr"_response']* called_csr['cnt_members']*.01).round()

called_csr.drop(['prop_ques_"called_csr"_response' ,'prop_called_aetna'], axis = 1,inplace=True )
called_csr = pd.DataFrame(called_csr.groupby('ques_called_csr_response').sum()).reset_index()

called_csr['prop_ques_"called_csr"_response'] = ((called_csr['cnt_members']/called_csr['cnt_members'].sum())*100).round(2)
called_csr['prop_called_aetna'] = ((called_csr['cnt_mbr_called_aetna']/called_csr['cnt_members'])*100).round(2)

col3 = list(list2[1].columns)
col3.remove('survey_month')
called_csr = called_csr[col3]


#Response

response = list2[4].copy()

response = pd.DataFrame(response.groupby('response').sum()).reset_index()

list_1 = ['doctor_search','review_coverage','received_eob','reviewed_claim','filled_prescription','called_csr','used_website']
for i in range(0,7):
    response[list_1[i]+'_mbr_cnt_prop'] = ((response[list_1[i]+'_mbr_cnt']/response[list_1[i]+'_mbr_cnt'].sum())*100).round(2)
    
#Rating

rating = list2[6].copy()
rating = pd.DataFrame(rating.groupby('question').sum()).reset_index()


#Task Sepcific Metrics

task_overall = list2[5].copy()
task_overall['customer_satisfaction'] = task_overall['avg_customer_satisfaction_score'] *task_overall['cnt_mbrs'] 
task_overall['paid_amt'] = task_overall['avg_paid_amt'] *task_overall['mbrs_clm_paid'] 


task_overall.drop(['avg_customer_satisfaction_score','avg_login_per_mbr', 'avg_den_clm_per_mem' , 'avg_paid_clm_per_mem','avg_paid_amt','median_paid_amt','avg_call_per_mem'],axis = 1,inplace=True )

task_overall = pd.DataFrame(task_overall.groupby('question').sum()).reset_index()

task_overall['avg_customer_satisfaction_score'] = task_overall['customer_satisfaction']/task_overall['cnt_mbrs']
task_overall['avg_login_per_mbr']= task_overall['login_count']/task_overall['mbrs_nav_login']
task_overall['avg_den_clm_per_mem']= task_overall['total_den_clm']/task_overall['mbrs_clm_den']
task_overall['avg_paid_clm_per_mem']= task_overall['total_paid_clm']/task_overall['mbrs_clm_paid']
task_overall['avg_paid_amt']= task_overall['paid_amt']/task_overall['mbrs_clm_paid']
task_overall['avg_call_per_mem']= task_overall['total_calls']/task_overall['total_mem_called']

col = list(list2[5].columns)
col.remove('median_paid_amt')
col.remove('survey_month')

task_overall = task_overall[col]

In [51]:
import time
TodaysDate = time.strftime("%d-%m-%Y")
excelfilename = "MED_REPORT_" + TodaysDate +".xlsx"

with pd.ExcelWriter(excelfilename) as writer:

    list2[0].to_excel(writer,sheet_name = 'med_csr_digital_interaction' , startrow = 1)
    list2[1].to_excel(writer,sheet_name = 'med_csr_digital_interaction' , startrow = len(list2[0]) + 3 )
    list2[2].to_excel(writer,sheet_name = 'med_csr_digital_interaction' , startrow = len(list2[0]) + len(list2[1]) + 5)
    list2[3].to_excel(writer,sheet_name = 'med_csr_digital_interaction' , startrow = len(list2[0]) + len(list2[1]) + 5, startcol = 5)
    
    list2[4].to_excel(writer,sheet_name = 'Response' , startrow = 1)
    list2[6].to_excel(writer,sheet_name = 'Response' , startrow = len(list2[4]) + 4 )

    
    list2[5].to_excel(writer,sheet_name = 'Task_specific_Metrices' , startrow = 1)
    
    list2[7].to_excel(writer,sheet_name = 'Top_speciality' , startrow = 1)
    
    #overall metrices
    
    #used_website.to_excel(writer,sheet_name = 'Month_consolidated', startrow = 1)
    #called_csr.to_excel(writer,sheet_name =  'Month_consolidated', startrow = len(used_website) + 3 )
    #response.to_excel(writer,sheet_name =  'Month_consolidated', startrow = len(used_website) + 5 + len(called_csr) )
    #rating.to_excel(writer,sheet_name =  'Month_consolidated', startrow = len(used_website) + 6 + len(response) + len ( called_csr))
    
    #task_overall.to_excel(writer,sheet_name = 'Month_consolidated_metrices', startrow = 1)

list2 = ['web','call_csr','top_reason','top_sub_reason','task_response','question','satisfaction','speciality']
for i in range(0,8):
    list2[i] = hc.table('cust_exp_enc.n323489_' + str(list2[i])).toPandas()
    
    Month = list(list2[5].survey_month.unique())
with pd.ExcelWriter('MED_REPORT.xlsx') as writer:

    list2[0].to_excel(writer,sheet_name = 'Med_data_validation' , startrow = 1)
    list2[1].to_excel(writer,sheet_name = 'Med_data_validation' , startrow = len(list2[0]) + 3 )
    list2[2].to_excel(writer,sheet_name = 'Med_data_validation' , startrow = len(list2[0]) + len(list2[1]) + 5)
    list2[3].to_excel(writer,sheet_name = 'Med_data_validation' , startrow = len(list2[0]) + len(list2[1]) + 5, startcol = 5)
    
    list2[4].to_excel(writer,sheet_name = 'Response' , startrow = 1)
    for i in range(0,len(Month)):  
        
        list2[6][list2[6]['survey_month'] == Month[i]].to_excel(writer,sheet_name = 'Response' , startrow = len(list2[4]) + 4 ,startcol = len(list2[5].columns)* i + 3 )

    
    for i in range(0,len(Month)):  
        list2[5][list2[5]['survey_month'] == Month[i]].to_excel(writer,sheet_name = 'Task_specific_Metrices' , startrow = 1,startcol = len(list2[5].columns)* i + 3)
      
        list2[7].to_excel(writer,sheet_name = 'Top_speciality' , startrow = 1)
    